In [1]:
pip install gdown

In [2]:
#!pip uninstall transformers -y

In [3]:
#!pip install -U adapter-transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00


In [4]:
pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.3 MB/s eta 0:00:00


In [5]:
pip install accelerate -U

In [6]:
import subprocess

# Define the URL of the Google Drive folder
folder_url = 'https://drive.google.com/drive/folders/1CAbb3DjrOPBNm0ozVBfhvrEh9P9rAppc'

# Command to download the folder using gdown
command = f'gdown --folder {folder_url}'

# Execute the command
subprocess.run(command, shell=True)

CompletedProcess(args='gdown --folder https://drive.google.com/drive/folders/1CAbb3DjrOPBNm0ozVBfhvrEh9P9rAppc', returncode=0)

In [7]:
from transformers import DebertaTokenizer, DebertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
#from evaluate import load as evaluate_load

In [8]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00


In [9]:
pip install datasets

In [10]:
pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [11]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging

In [12]:
import re
import numpy as np
from scipy.special import softmax
import sentencepiece  # Ensure this import is included
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
from evaluate import load as evaluate_load

In [13]:
from transformers import TrainingArguments, Trainer, EvalPrediction

In [14]:
train_df=pd.read_json('/content/SubtaskA/subtaskA_train_monolingual.jsonl',lines=True)
train_df.head()

,text,label,model,source,id
0,Forza Motorsport is a popular racing game that...,1,chatGPT,wikihow,0
1,Buying Virtual Console games for your Nintendo...,1,chatGPT,wikihow,1
2,Windows NT 4.0 was a popular operating system ...,1,chatGPT,wikihow,2
3,How to Make Perfume\n\nPerfume is a great way ...,1,chatGPT,wikihow,3
4,How to Convert Song Lyrics to a Song'\n\nConve...,1,chatGPT,wikihow,4


In [15]:
train_df

,text,label,model,source,id
0,Forza Motorsport is a popular racing game that...,1,chatGPT,wikihow,0
1,Buying Virtual Console games for your Nintendo...,1,chatGPT,wikihow,1
2,Windows NT 4.0 was a popular operating system ...,1,chatGPT,wikihow,2
3,How to Make Perfume\n\nPerfume is a great way ...,1,chatGPT,wikihow,3
4,How to Convert Song Lyrics to a Song'\n\nConve...,1,chatGPT,wikihow,4
...,...,...,...,...,...
119752,"The paper is an interesting contribution, prim...",0,human,peerread,119752
119753,\nWe thank the reviewers for all their comment...,0,human,peerread,119753
119754,The authors introduce a semi-supervised method...,0,human,peerread,119754
119755,This paper proposes the Neural Graph Machine t...,0,human,peerread,119755


In [16]:
'''
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Visualization 1: Bar Chart of Class Distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='label', data=train_df)
plt.title('Class Distribution')
plt.show()
'''



"\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n\n# Visualization 1: Bar Chart of Class Distribution\nplt.figure(figsize=(8, 6))\nsns.countplot(x='label', data=train_df)\nplt.title('Class Distribution')\nplt.show()\n"

In [17]:
'''
from wordcloud import WordCloud
machine = ' '.join(train_df[train_df['label'] == 1]['text'])
human = ' '.join(train_df[train_df['label'] == 0]['text'])

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
wordcloud_positive = WordCloud(width=400, height=400, background_color='white').generate(machine)
plt.imshow(wordcloud_positive, interpolation='bilinear')
plt.title('Word Cloud - Machine')

plt.subplot(1, 2, 2)
wordcloud_negative = WordCloud(width=400, height=400, background_color='white').generate(human)
plt.imshow(wordcloud_negative, interpolation='bilinear')
plt.title('Word Cloud - Human')

plt.show()
'''

"\nfrom wordcloud import WordCloud\nmachine = ' '.join(train_df[train_df['label'] == 1]['text'])\nhuman = ' '.join(train_df[train_df['label'] == 0]['text'])\n\nplt.figure(figsize=(12, 6))\n\nplt.subplot(1, 2, 1)\nwordcloud_positive = WordCloud(width=400, height=400, background_color='white').generate(machine)\nplt.imshow(wordcloud_positive, interpolation='bilinear')\nplt.title('Word Cloud - Machine')\n\nplt.subplot(1, 2, 2)\nwordcloud_negative = WordCloud(width=400, height=400, background_color='white').generate(human)\nplt.imshow(wordcloud_negative, interpolation='bilinear')\nplt.title('Word Cloud - Human')\n\nplt.show()\n"

In [18]:
'''
plt.figure(figsize=(10, 6))
sns.histplot(train_df, x=train_df['text'].apply(len), hue='label', bins=30, kde=True)
plt.title('Distribution of Text Length by Class')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.show()
'''

"\nplt.figure(figsize=(10, 6))\nsns.histplot(train_df, x=train_df['text'].apply(len), hue='label', bins=30, kde=True)\nplt.title('Distribution of Text Length by Class')\nplt.xlabel('Text Length')\nplt.ylabel('Frequency')\nplt.show()\n"

In [19]:
'''
plt.figure(figsize=(10, 6))
sns.histplot(train_df, x=train_df['text'].apply(len), hue='label', bins=30, kde=True)
plt.title('Distribution of Text Length by Class')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.xlim(0, 25000)  # Set x-axis limit to 500
plt.show()
'''

"\nplt.figure(figsize=(10, 6))\nsns.histplot(train_df, x=train_df['text'].apply(len), hue='label', bins=30, kde=True)\nplt.title('Distribution of Text Length by Class')\nplt.xlabel('Text Length')\nplt.ylabel('Frequency')\nplt.xlim(0, 25000)  # Set x-axis limit to 500\nplt.show()\n"

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Specify the features (X) and the label (y)
X = train_df.drop('label', axis=1)  # Features
y = train_df['label']  # Label

# Split the data into training and testing sets (70:30 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create new DataFrames for training and testing sets
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)


In [21]:
len(train_df)

83829

In [22]:
len(test_df)

35928

In [23]:
df_label_0 = train_df[train_df['label'] == 0].sample(10000)
df_label_1 = train_df[train_df['label'] == 1].sample(10000)

# Concatenating the two subsets to get the final balanced dataset
train_df = pd.concat([df_label_0, df_label_1])

In [24]:
df_label_0_t = test_df[test_df['label'] == 0].sample(1000)
df_label_1_t = test_df[test_df['label'] == 1].sample(1000)

# Concatenating the two subsets to get the final balanced dataset
test_df = pd.concat([df_label_0_t, df_label_1_t])

In [25]:
len(train_df)

20000

In [26]:
len(test_df)

2000

In [27]:
valid_df=pd.read_json('/content/SubtaskA/subtaskA_dev_monolingual.jsonl',lines=True)
valid_df.head()

,text,label,model,source,id
0,Giving gifts should always be enjoyable. Howe...,1,bloomz,wikihow,0
1,Yveltal (Japanese: ユベルタル) is one of the main a...,1,bloomz,wikihow,1
2,If you'd rather not annoy others by being rude...,1,bloomz,wikihow,2
3,If you're interested in visiting gravesite(s) ...,1,bloomz,wikihow,3
4,The following are some tips for becoming succe...,1,bloomz,wikihow,4


In [28]:
df_label_0_v = valid_df[valid_df['label'] == 0].sample(2000)
df_label_1_v = valid_df[valid_df['label'] == 1].sample(2000)

# Concatenating the two subsets to get the final balanced dataset
valid_df = pd.concat([df_label_0_v, df_label_1_v])

In [29]:
len(valid_df)

4000

In [30]:
import re
import numpy as np
from scipy.special import softmax
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
from evaluate import load as evaluate_load


In [31]:
'''
def preprocess_function(examples, **fn_kwargs):
    number_pattern = r'\d+'  # Pattern to match numbers
    html_pattern = r'<.*?>'  # Pattern to match HTML tags
    extra_space_pattern = r'\s+'  # Pattern to match one or more whitespace characters

    # Function to remove numbers, HTML tags, and extra spaces
    def clean_text(text):
        text = re.sub(number_pattern, '', text)  # Remove numbers
        text = re.sub(html_pattern, '', text)    # Remove HTML tags
        text = re.sub(extra_space_pattern, ' ', text)  # Replace extra spaces with a single space
        return text.strip()  # Remove leading and trailing spaces

    # Apply the cleaning function to each example
    cleaned_texts = [clean_text(text) for text in examples["text"]]

    # Tokenize the cleaned texts
    #return fn_kwargs['tokenizer'](cleaned_texts, truncation=True)
    #return cleaned_texts
    return fn_kwargs['tokenizer'](cleaned_texts, truncation=True)

'''

'\ndef preprocess_function(examples, **fn_kwargs):\n    number_pattern = r\'\\d+\'  # Pattern to match numbers\n    html_pattern = r\'<.*?>\'  # Pattern to match HTML tags\n    extra_space_pattern = r\'\\s+\'  # Pattern to match one or more whitespace characters\n\n    # Function to remove numbers, HTML tags, and extra spaces\n    def clean_text(text):\n        text = re.sub(number_pattern, \'\', text)  # Remove numbers\n        text = re.sub(html_pattern, \'\', text)    # Remove HTML tags\n        text = re.sub(extra_space_pattern, \' \', text)  # Replace extra spaces with a single space\n        return text.strip()  # Remove leading and trailing spaces\n\n    # Apply the cleaning function to each example\n    cleaned_texts = [clean_text(text) for text in examples["text"]]\n\n    # Tokenize the cleaned texts\n    #return fn_kwargs[\'tokenizer\'](cleaned_texts, truncation=True)\n    #return cleaned_texts\n    return fn_kwargs[\'tokenizer\'](cleaned_texts, truncation=True)\n\n'

In [32]:
import re

def preprocess_function(examples, tokenizer):
    number_pattern = r'\d+'  # Pattern to match numbers
    html_pattern = r'<.*?>'  # Pattern to match HTML tags
    extra_space_pattern = r'\s+'  # Pattern to match one or more whitespace characters

    # Function to remove numbers, HTML tags, and extra spaces
    def clean_text(text):
        text = re.sub(number_pattern, '', text)  # Remove numbers
        text = re.sub(html_pattern, '', text)    # Remove HTML tags
        text = re.sub(extra_space_pattern, ' ', text)  # Replace extra spaces with a single space
        return text.strip()  # Remove leading and trailing spaces

    # Apply the cleaning function to each example
    cleaned_texts = [clean_text(text) for text in examples["text"]]

    # Tokenize the cleaned texts
    return tokenizer(cleaned_texts, truncation=True, padding='max_length', max_length=512)

# Usage example in fine_tune or test function
# tokenized_train_dataset = train_dataset.map(lambda x: preprocess_function(x, tokenizer=tokenizer), batched=True)


In [33]:
'''
# Example usage
#model_name = "roberta-base"
#tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assuming you have a DataFrame 'train_df' with a column 'text'
train_df_1 = train_df[:15]  # Your DataFrame
train_dataset = Dataset.from_pandas(train_df_1)

# Apply the preprocessing function to the dataset
tokenized_train_dataset = preprocess_function(train_dataset)
'''

'\n# Example usage\n#model_name = "roberta-base"\n#tokenizer = AutoTokenizer.from_pretrained(model_name)\n\n# Assuming you have a DataFrame \'train_df\' with a column \'text\'\ntrain_df_1 = train_df[:15]  # Your DataFrame\ntrain_dataset = Dataset.from_pandas(train_df_1)\n\n# Apply the preprocessing function to the dataset\ntokenized_train_dataset = preprocess_function(train_dataset)\n'

In [34]:
#tokenized_train_dataset

In [35]:
#tokenized_train_dataset.to_csv

In [36]:
'''tokenizer = AutoTokenizer.from_pretrained(model)     # put your model here
model = AutoModelForSequenceClassification.from_pretrained(
       model, num_labels=len(label2id), id2label=id2label, label2id=label2id    # put your model here
    )

    # tokenize data for train/valid
tokenized_train_dataset = train_df.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
tokenized_valid_dataset = valid_df.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})'''

"tokenizer = AutoTokenizer.from_pretrained(model)     # put your model here\nmodel = AutoModelForSequenceClassification.from_pretrained(\n       model, num_labels=len(label2id), id2label=id2label, label2id=label2id    # put your model here\n    )\n\n    # tokenize data for train/valid\ntokenized_train_dataset = train_df.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})\ntokenized_valid_dataset = valid_df.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})"

In [37]:
def get_data(train_path, test_path, random_seed):
    """
    function to read dataframe with columns
    """

    train_df = pd.read_json(train_path, lines=True)
    test_df = pd.read_json(test_path, lines=True)

    train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=random_seed)

    return train_df, val_df, test_df


In [38]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

In [39]:
#Trainer = AdapterTrainer

In [40]:
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaConfig
#, RobertaModelWithHeads
from datasets import Dataset
from transformers import RobertaConfig
#, RobertaModelWithHeads

In [41]:
import nltk

In [42]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [43]:
def compute_metrics(eval_pred):
    f1_metric = evaluate_load("f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1_metric.compute(predictions=predictions, references=labels, average="micro")

def fine_tune(train_df, valid_df, checkpoints_path, id2label, label2id, model):

    # pandas dataframe to huggingface Dataset
    train_dataset = Dataset.from_pandas(train_df)
    valid_dataset = Dataset.from_pandas(valid_df)

    # get tokenizer and model from huggingface
         # put your model here
    tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base')
    model = AutoModelForSequenceClassification.from_pretrained(
       model, num_labels=len(label2id), id2label=id2label, label2id=label2id    # put your model here
    )

    optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
    lr_scheduler = AdafactorSchedule(optimizer)

    # tokenize data for train/valid
    tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
    tokenized_valid_dataset = valid_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #model.to(device)
    #from transformers import RobertaConfig, RobertaModelWithHeads

    #Defining the configuration for the model

    #Setting up the model
    #model = RobertaModelWithHeads.from_pretrained("roberta-base", config=config)

    # create Trainer
    training_args = TrainingArguments(
        output_dir=checkpoints_path,
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_valid_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        optimizers=(optimizer, lr_scheduler),
    )

    trainer.train()

    # save best model
    best_model_path = checkpoints_path+'/best/'

    if not os.path.exists(best_model_path):
        os.makedirs(best_model_path)


    trainer.save_model(best_model_path)


def test(test_df, model_path, id2label, label2id):

    # load tokenizer from saved model
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # load best model
    model = AutoModelForSequenceClassification.from_pretrained(
       model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id
    )

    test_dataset = Dataset.from_pandas(test_df)

    tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #model.to(device)

    # create Trainer
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    # get logits from predictions and evaluate results using classification report
    predictions = trainer.predict(tokenized_test_dataset)
    prob_pred = softmax(predictions.predictions, axis=-1)
    preds = np.argmax(predictions.predictions, axis=-1)
    metric = evaluate.load("bstrai/classification_report")
    results = metric.compute(predictions=preds, references=predictions.label_ids)

    # return dictionary of classification report
    return results, preds

In [44]:
id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1}

In [45]:
model='roberta-base'

In [46]:
subtask='A'

In [47]:
random_seed=42

In [48]:
from transformers import get_scheduler

In [49]:
import os
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AdapterType, RobertaConfig, RobertaModelWithHeads
from datasets import Dataset

In [50]:
from transformers.optimization import Adafactor, AdafactorSchedule
#from transformers.optimization import AdamWfactor, AdamWSchedule

In [51]:
from transformers import get_scheduler

In [52]:
import torch

In [53]:
fine_tune(train_df[['text', 'label']], valid_df[['text', 'label']], f"{model}/subtask{subtask}/{random_seed}", id2label, label2id, model)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.502200,0.775404,0.556500
2,0.701600,0.694439,0.500000
3,0.695000,0.697462,0.500000
4,0.694200,0.693367,0.500000
5,0.693400,0.693982,0.500000


In [54]:
'''
if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument("--train_file_path", "-tr", required=True, help="Path to the train file.", type=str)
    parser.add_argument("--test_file_path", "-t", required=True, help="Path to the test file.", type=str)
    parser.add_argument("--subtask", "-sb", required=True, help="Subtask (A or B).", type=str, choices=['A', 'B'])
    parser.add_argument("--model", "-m", required=True, help="Transformer to train and test", type=str)
    parser.add_argument("--prediction_file_path", "-p", required=True, help="Path where to save the prediction file.", type=str)

    args = parser.parse_args()

    random_seed = 0
    train_path =  '/content/SubtaskA/subtaskA_train_monolingual.jsonl' # For example 'subtaskA_train_multilingual.jsonl'
    test_path =  '/content/SubtaskA/subtaskA_dev_monolingual.jsonl' # For example 'subtaskA_test_multilingual.jsonl'
    model =  args.model # For example 'xlm-roberta-base'
    subtask =  args.subtask # For example 'A'
    prediction_path = args.prediction_file_path # For example subtaskB_predictions.jsonl

    if not os.path.exists(train_path):
        logging.error("File doesnt exists: {}".format(train_path))
        raise ValueError("File doesnt exists: {}".format(train_path))

    if not os.path.exists(test_path):
        logging.error("File doesnt exists: {}".format(train_path))
        raise ValueError("File doesnt exists: {}".format(train_path))


    if subtask == 'A':
        id2label = {0: "human", 1: "machine"}
        label2id = {"human": 0, "machine": 1}
    elif subtask == 'B':
        id2label = {0: 'human', 1: 'chatGPT', 2: 'cohere', 3: 'davinci', 4: 'bloomz', 5: 'dolly'}
        label2id = {'human': 0, 'chatGPT': 1,'cohere': 2, 'davinci': 3, 'bloomz': 4, 'dolly': 5}
    else:
        logging.error("Wrong subtask: {}. It should be A or B".format(train_path))
        raise ValueError("Wrong subtask: {}. It should be A or B".format(train_path))

    set_seed(random_seed)

    #get data for train/dev/test sets
    train_df, valid_df, test_df = get_data(train_path, test_path, random_seed)

    # train detector model
    fine_tune(train_df, valid_df, f"{model}/subtask{subtask}/{random_seed}", id2label, label2id, model)

    # test detector model
    results, predictions = test(test_df, f"{model}/subtask{subtask}/{random_seed}/best/", id2label, label2id)

    logging.info(results)
    predictions_df = pd.DataFrame({'id': test_df['id'], 'label': predictions})
    predictions_df.to_json(prediction_path, lines=True, orient='records')
'''

'\nif __name__ == \'__main__\':\n\n    parser = argparse.ArgumentParser()\n    parser.add_argument("--train_file_path", "-tr", required=True, help="Path to the train file.", type=str)\n    parser.add_argument("--test_file_path", "-t", required=True, help="Path to the test file.", type=str)\n    parser.add_argument("--subtask", "-sb", required=True, help="Subtask (A or B).", type=str, choices=[\'A\', \'B\'])\n    parser.add_argument("--model", "-m", required=True, help="Transformer to train and test", type=str)\n    parser.add_argument("--prediction_file_path", "-p", required=True, help="Path where to save the prediction file.", type=str)\n\n    args = parser.parse_args()\n\n    random_seed = 0\n    train_path =  \'/content/SubtaskA/subtaskA_train_monolingual.jsonl\' # For example \'subtaskA_train_multilingual.jsonl\'\n    test_path =  \'/content/SubtaskA/subtaskA_dev_monolingual.jsonl\' # For example \'subtaskA_test_multilingual.jsonl\'\n    model =  args.model # For example \'xlm-robe

In [55]:
#python your_script.py --train_file_path /content/SubtaskA/subtaskA_train_monolingual.jsonl --test_file_path /content/SubtaskA/subtaskA_dev_monolingual.jsonl --subtask A --model your_model_name --prediction_file_path /path/to/save/predictions


In [56]:
results, predictions = test(test_df[['text', 'label']], f"{model}/subtask{subtask}/{random_seed}/best/", id2label, label2id)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
results

{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1000},
 '1': {'precision': 0.5,
  'recall': 1.0,
  'f1-score': 0.6666666666666666,
  'support': 1000},
 'accuracy': 0.5,
 'macro avg': {'precision': 0.25,
  'recall': 0.5,
  'f1-score': 0.3333333333333333,
  'support': 2000},
 'weighted avg': {'precision': 0.25,
  'recall': 0.5,
  'f1-score': 0.3333333333333333,
  'support': 2000}}

In [58]:
logging.info(results)
predictions_df = pd.DataFrame({'id': test_df['id'], 'label': predictions})

In [59]:
predictions_df

,id,label
104636,104636,1
112103,112103,1
69133,69133,1
84189,84189,1
75086,75086,1
...,...,...
35986,35986,1
12551,12551,1
26095,26095,1
29800,29800,1


In [60]:
predictions_df.to_json('/content/prediction', lines=True, orient='records')